In [ ]:
import pygame
import time
# possibly need to set export DISPLAY=:0 in terminal?

# does this need to run?
# pygame.display.list_modes()

## TO RUN VIDEOSTUFF:
- 1: start xming
- 2: in terminal launching notebook, run export DISPLAY=:0

In [ ]:
SCREEN_HEIGHT

In [ ]:
CELL_RADIUS

In [ ]:

NUM_ROWS = 6
NUM_COLS = 7

CELL_RADIUS = 20
CELL_PADDING = 15

SCREEN_HEIGHT = NUM_ROWS * (2*CELL_RADIUS + CELL_PADDING) + 2*CELL_PADDING
SCREEN_WIDTH = NUM_COLS * (2*CELL_RADIUS + CELL_PADDING) + 2*CELL_PADDING

VELOCITY = 1

BOARD_COLOR = pygame.Color('green')
CELL_COLOR = pygame.Color('black')


CELL_PADDING

In [ ]:
pygame.init()

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) # returns the surface where you can draw stuff

pygame.draw.rect(screen, BOARD_COLOR, pygame.Rect((0,0), (SCREEN_WIDTH,SCREEN_HEIGHT)))


# draw empty cells

y_pos = CELL_RADIUS + CELL_PADDING
for _ in range(NUM_ROWS):
    x_pos = CELL_RADIUS + CELL_PADDING
    for _ in range(NUM_COLS):
        pygame.draw.circle(screen, CELL_COLOR, (x_pos, y_pos), CELL_RADIUS)
        x_pos += 2 * CELL_RADIUS + CELL_PADDING
    y_pos += 2 * CELL_RADIUS + CELL_PADDING

    #break

#pygame.draw.circle(screen, CELL_COLOR, (SCREEN_WIDTH/2, SCREEN_HEIGHT/2), CELL_RADIUS)

# run game

pygame.display.flip()
time.sleep(1)
while True:
    e = pygame.event.poll()
    if e.type == pygame.QUIT:
        break
        #time.sleep(10)
    pygame.display.flip()
    ballplay.update()
        
pygame.quit()

In [ ]:
y_pos

In [ ]:
xpos

## OLD PONG STUFF

In [ ]:
class Ball:
    
    RADIUS = 20
    
    def __init__(self, x, y, vx, vy):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        
    def show(self, color):
        global screen # take screen from global vars        
        pygame.draw.circle(screen, color, (self.x, self.y), self.RADIUS)
    
    def update(self):
        global ballColor, bgColor
        self.show(bgColor) # hides the current ball
        newx = self.x + self.vx
        newy = self.y + self.vy
        
        if newx < BORDER + self.RADIUS:
            self.vx = -self.vx
        elif newy < BORDER + self.RADIUS or newy > SCREEN_HEIGHT - (BORDER + self.RADIUS):
            self.vy = -self.vy
        else:
            self.x = newx
            self.y = newy
            self.show(ballColor)
         
class Paddle:
    PADDLE_WIDTH = 20
    PADDÖE_HEIGHT = 100

    def __init__(self, y):
        self.y = y

    def show(self, color):
        global screen, paddleColor
        pygame.draw.rect(screen, paddleColor, )


In [ ]:
pygame.init()

SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 600
BORDER = 20
VELOCITY = 1

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT)) # returns the surface where you can draw stuff

fgColor = pygame.Color('green')
bgColor = pygame.Color('black')
ballColor = pygame.Color('red')
paddleColor = pygame.Color('yellow')

# draw the borders
pygame.draw.rect(screen, fgColor, pygame.Rect((0,0), (SCREEN_WIDTH,BORDER)))
pygame.draw.rect(screen, fgColor, pygame.Rect((0,0), (BORDER, SCREEN_HEIGHT)))
pygame.draw.rect(screen, fgColor, pygame.Rect((0, SCREEN_HEIGHT-BORDER), (SCREEN_WIDTH,BORDER)))

# draw the ball
ballplay = Ball(SCREEN_WIDTH-Ball.RADIUS, SCREEN_HEIGHT // 2, -VELOCITY, VELOCITY)
ballplay.show(ballColor)

pygame.display.flip()

time.sleep(10)

while True:
    e = pygame.event.poll()
    if e.type == pygame.QUIT:
        break
        #time.sleep(10)

    pygame.display.flip()
    ballplay.update()
        
pygame.quit()

In [ ]:
pygame.quit()

In [ ]:
help(pygame.draw.rect)

In [ ]:
import numpy 

Notes -- pygame / graphics with WSL

WSL stödjer inte grafiska saker (linux kör X11)

Dock finns något som heter Xming som gör så att X11 funkar inuti windows(?)
- https://stackoverflow.com/questions/53124539/pygame-video-not-initialized-and-no-video-found-using-ubuntu-terminal-from-wi

X-ming downloads: https://sourceforge.net/projects/xming/
